# Inhibitory Neuron Analysis

Create manifest to keep track of data

In [5]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import pprint

# This class uses a 'manifest' to keep track of downloaded data and metadata.  
# All downloaded files will be stored relative to the directory holding the manifest
# file.  If 'manifest_file' is a relative path (as it is below), it will be 
# saved relative to your working directory.  It can also be an absolute path.
boc = BrainObservatoryCache()

In [6]:
from ipynb.fs.defs.drifting_gratings import create_histogram

In [7]:
# Download a list of all cre driver lines 
cre_lines = boc.get_all_cre_lines()
print("all cre lines:\n")
pprint.pprint(cre_lines)

all cre lines:

['Cux2-CreERT2',
 'Emx1-IRES-Cre',
 'Fezf2-CreER',
 'Nr5a1-Cre',
 'Ntsr1-Cre_GN220',
 'Pvalb-IRES-Cre',
 'Rbp4-Cre_KL100',
 'Rorb-IRES2-Cre',
 'Scnn1a-Tg3-Cre',
 'Slc17a7-IRES2-Cre',
 'Sst-IRES-Cre',
 'Tlx3-Cre_PL56',
 'Vip-IRES-Cre']


In [14]:
# Download experiment containers for Sst and Vip experiments
Sst_ecs = boc.get_experiment_containers(cre_lines=['Sst-IRES-Cre'])
print("Sst experiments: %d\n" % len(Sst_ecs))

print("Example experiment container record:")
pprint.pprint(Sst_ecs[0])
print("\n")


Vip_ecs = boc.get_experiment_containers(cre_lines=['Vip-IRES-Cre'])
print("Vip experiments: %d\n" % len(Vip_ecs))

print("Example experiment container record:")
pprint.pprint(Vip_ecs[0])



Sst experiments: 64

Example experiment container record:
{'cre_line': 'Sst-IRES-Cre',
 'donor_name': '306500',
 'failed': False,
 'id': 660492886,
 'imaging_depth': 275,
 'reporter_line': 'Ai148(TIT2L-GC6f-ICL-tTA2)',
 'specimen_name': 'Sst-IRES-Cre;Ai148(CAM)-306500',
 'tags': [],
 'targeted_structure': 'VISal'}


Vip experiments: 76

Example experiment container record:
{'cre_line': 'Vip-IRES-Cre',
 'donor_name': '336246',
 'failed': False,
 'id': 617388115,
 'imaging_depth': 175,
 'reporter_line': 'Ai148(TIT2L-GC6f-ICL-tTA2)',
 'specimen_name': 'Vip-IRES-Cre;Ai148-336246',
 'tags': [],
 'targeted_structure': 'VISpm'}
